In [ ]:
import pandas as pd
import re

In [ ]:
data = pd.read_csv("./data/spotify_songs.csv")

In [ ]:
print(data.info())

In [ ]:
def clean_text(lyric):
     return  re.sub(r'[^a-zA-Z\s]', '', lyric)

In [ ]:
data = data[data['language'] == 'en']
data['lyrics'] = data['lyrics'].apply(clean_text)
data['track_name'] = data['track_name'].apply(clean_text)
data['track_album_name'] = data['track_album_name'].apply(clean_text)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [ ]:
def sentiment_score(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    
    with torch.no_grad():
        outputs = model(**inputs)

    scores = outputs[0][0].softmax(0)
    scores = scores.detach().numpy()
    max_score = scores.argmax()
    
    return max_score 

In [ ]:
data['lyrics_sentiment'] = data['lyrics'].apply(sentiment_score)

In [ ]:
data['album_name_sentiment'] = data['track_album_name'].apply(sentiment_score)

In [ ]:
data['track_name_sentiment'] = data['track_name'].apply(sentiment_score)

In [ ]:
data.to_csv("spotify_songs_processed.csv", index=False)